# 注意
<hr>
本程式是在Win10環境執行，<br>
使用jupyter notebook編寫。<br>

主目錄中要有pytorch_object_detection中的資料匣，<br>
cell會呼叫pytorch_object_detection中的程式。<br>

訓練整個Train資料匣，需要花費 22 小時，<br>
訓練時請確定vram有7G以上。<br>

GPU: RTX-2070 Super<br>
CPU: i7-10700<br>
RAM: 16G<br>
Python:3.8.6<br>
torch:1.6.0<br>
torchvision:0.7.0<br>

為了在不同電腦上都順利執行，<br>
本程式cell中的路徑都為相對路徑非絕對路徑，<br>
重複執行同個cell將會出現錯誤。

**請將下面(約程式中段)的cell取消comment，以進行訓練**

In [ ]:
!pip install pycocotools
!pip install tqdm

## Getting data

若是發生路徑問題，請確認pwd在正確位置

In [1]:
pwd

'D:\\user\\Documents\\N26091194_TENG\\Assignments\\NCKU_ML\\mango'

### processing data

In [2]:
import numpy as np
import pandas as pd
label=[]
image=[]
xmin=[]
ymin=[]
xmax=[]
ymax=[]

with open(r'.\C2_TrainDev\train.csv', 'r', encoding='utf-8-sig') as fh:
  for index,row in enumerate(fh):
    row=row.strip()
    row=row.split(',')
    while True:
      if '' in row:
        row.remove('')
      else:
        break

    while len(row) > 1:
      xmin.append(float(row[1]))
      ymin.append(float(row[2]))
      xmax.append(float(row[3])+float(row[1]))
      ymax.append(float(row[4])+float(row[2]))
      label.append(row[5])
      for idx in range(5):
        row.remove(row[1]) 
      image.append(row[0])

traindf = pd.DataFrame({'image':image, 'label':label, 'xmin':xmin, 'ymin':ymin, 'xmax':xmax, 'ymax':ymax  })
traindf.to_pickle(r".\C2_TrainDev\traindf.pkl")

## Pytorch

In [3]:
import pycocotools

In [4]:
import os
try:
    os.mkdir(r".\pytorch_object_detection")
    print('created folder "pytorch_object_detection"')
except:
    print('"pytorch_object_detection" folder already exists')
    pass

"pytorch_object_detection" folder already exists


In [5]:
cd .\pytorch_object_detection

D:\user\Documents\N26091194_TENG\Assignments\NCKU_ML\mango\pytorch_object_detection


In [6]:
!git clone https://github.com/pytorch/vision.git

fatal: destination path 'vision' already exists and is not an empty directory.


In [7]:
cd .\vision\references\detection

D:\user\Documents\N26091194_TENG\Assignments\NCKU_ML\mango\pytorch_object_detection\vision\references\detection


In [8]:
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import pandas as pd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision
from engine import train_one_epoch, evaluate
import utils
import transforms as T
from tqdm import tqdm
from PIL import Image
from PIL import ImageDraw

### Configuring the model

#### 1. Preparing the data

In [9]:
def parse_one_annot(image, train_state=True):
  if train_state:
    boxes_array = traindf[traindf["image"] == image][["xmin", "ymin", "xmax", "ymax"]].values
  else:
    boxes_array = validdf[validdf["image"] == image][["xmin", "ymin", "xmax", "ymax"]].values
  return boxes_array

##### training set

In [10]:
class TrainDataset(torch.utils.data.Dataset):
  def __init__(self, root, transforms=None):
    self.root = root
    self.transforms = transforms
    self.imgs = sorted(os.listdir(os.path.join(root, "Train")))

  def __getitem__(self, idx):
    # load images and bounding boxes
    img_path = os.path.join(self.root, "Train", self.imgs[idx])
    img = Image.open(img_path).convert("RGB")
    box_list = parse_one_annot(self.imgs[idx], True)
    boxes = torch.as_tensor(box_list, dtype=torch.float32)
    num_objs = len(box_list)

    # there is only one class******************************************************
    labels = torch.ones((num_objs,), dtype=torch.int64)
    for row in range(len(traindf[traindf['image'] == self.imgs[idx]])):
      if traindf[traindf['image'] == self.imgs[idx]]['label'].iloc[row] == '不良-乳汁吸附':
        labels[row] = 1
      elif traindf[traindf['image'] == self.imgs[idx]]['label'].iloc[row] == '不良-機械傷害':
        labels[row] = 2
      elif traindf[traindf['image'] == self.imgs[idx]]['label'].iloc[row] == '不良-炭疽病': 
        labels[row] = 3
      elif traindf[traindf['image'] == self.imgs[idx]]['label'].iloc[row] == '不良-著色不佳':
        labels[row] = 4
      else:
        labels[row] = 5
      
    image_id = torch.tensor([idx])
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:,0])

    # suppose all instances are not crowd
    iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
    target = {}
    target["boxes"] = boxes
    target["labels"] = labels
    target["image_id"] = image_id
    target["area"] = area
    target["iscrowd"] = iscrowd
    if self.transforms is not None:
      img, target = self.transforms(img, target)
    return img, target

  def __len__(self):
    return len(self.imgs)

##### valid set

In [11]:
class ValidDataset(torch.utils.data.Dataset):
  def __init__(self, root, transforms=None):
    self.root = root
    self.transforms = transforms
    self.imgs = sorted(os.listdir(os.path.join(root, "Dev")))

  def __getitem__(self, idx):
    # load images and bounding boxes
    img_path = os.path.join(self.root, "Dev", self.imgs[idx])
    img = Image.open(img_path).convert("RGB")
    target = {}
    if self.transforms is not None:
      img, target = self.transforms(img, target)
    return img, target

  def __len__(self):
    return len(self.imgs)

In [12]:
train_set = TrainDataset(root= ".\..\..\..\..\C2_TrainDev")
valid_set = ValidDataset(root= r".\..\..\..\..\C2_TrainDev")

#### 2.Download and adjust the model

In [13]:
def get_model(num_classes):
  # load an object detection model pre-trained on COCO
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  # get the number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # replace the pre-trained head with a new on
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)
   
  return model

In [14]:
def get_transform(train):
  transforms = []
  # converts the image, a PIL image, into a PyTorch Tensor
  transforms.append(T.ToTensor())
  if train:
    # during training, randomly flip the training images
    # and ground-truth for data augmentation
    transforms.append(T.RandomHorizontalFlip(0.5))
  return T.Compose(transforms)

In [15]:
# use our dataset and defined transformations
train_set = TrainDataset(root= r".\..\..\..\..\C2_TrainDev",transforms = get_transform(train=True))

valid_set = ValidDataset(root= r".\..\..\..\..\C2_TrainDev", transforms = get_transform(train=False))
# split the dataset in train and test set
torch.manual_seed(1)

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle=True, collate_fn=utils.collate_fn)
#data_loader_test = torch.utils.data.DataLoader(valid_set, batch_size=1, shuffle=False, collate_fn=utils.collate_fn)
#print("We have: {} examples,  {} are training and {} testing".format(len(train_set)+len(valid_set) ,len(train_set), len(valid_set)))

In [16]:
if torch.cuda.is_available():
    print('GPU:',torch.cuda.get_device_name())
    print('-Training on GPU-')
else:
    print('GPU is not avalible!')
    print('-Training on CPU-')

GPU is not avalible!
-Training on CPU-


In [17]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# our dataset has six classes only 
num_classes = 6
# get the model using our helper function
model = get_model(num_classes)
# move model to the right device
model.to(device)
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler which decreases the learning rate by # 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# 注意
<hr>
若要自行**訓練**，請將下列cell 取消comment。<br>
請確定目前VRAM有7G以上空間。<br>
本模型在Dev_set上需訓練22小時。<br>

**請確保執行期間程式不會中斷。**

In [18]:
num_epochs = 1
for epoch in range(num_epochs):
   # train for one epoch, printing every 10 iterations
   train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10) 
   
   lr_scheduler.step() # update the learning rate
   

try:
    os.mkdir(r".\..\..\..\..\pytorch_object_detection\mango")
except:
    print('folder already exists')
    pass
torch.save(model.state_dict(), r".\..\..\..\..\pytorch_object_detection\mango\model")


c:\users\user\appdata\local\programs\python\python38\lib\site-packages\torchvision\ops\boxes.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:766.)
  keep = keep.nonzero().squeeze(1)


Epoch: [0]  [ 0/10]  eta: 0:01:07  lr: 0.000560  loss: 3.1001 (3.1001)  loss_classifier: 2.7666 (2.7666)  loss_box_reg: 0.0438 (0.0438)  loss_objectness: 0.2722 (0.2722)  loss_rpn_box_reg: 0.0175 (0.0175)  time: 6.7515  data: 0.0409
Epoch: [0]  [ 9/10]  eta: 0:00:06  lr: 0.005000  loss: 0.5879 (1.3398)  loss_classifier: 0.3951 (1.0232)  loss_box_reg: 0.0058 (0.0261)  loss_objectness: 0.1047 (0.2774)  loss_rpn_box_reg: 0.0076 (0.0131)  time: 6.7946  data: 0.0318
Epoch: [0] Total time: 0:01:07 (6.7947 s / it)
folder already exists


## Loading model

In [19]:
loaded_model = get_model(num_classes = 6)
loaded_model.load_state_dict(torch.load(r".\..\..\..\..\pytorch_object_detection\mango\model", map_location=device))

<All keys matched successfully>

## Output Prediction

In [20]:
root= r".\..\..\..\..\C2_TrainDev"
imgs = sorted(os.listdir(os.path.join(root, "Dev")))

# 注意
<hr>
預測完整個Dev資料匣需要耗費3小時<br>

**請確定執行過程不會中斷**

In [21]:
loaded_model.eval()
files=[]
#label_map = {'不良-乳汁吸附':1, '不良-機械傷害':2, '不良-炭疽病':3, '不良-著色不佳':4, '不良-黑斑病':5}
class1=[]
class2=[]
class3=[]
class4=[]
class5=[]

for i in tqdm(range(len(imgs)), position=0, leave=True):
    idx = i
    img, _ = valid_set[idx]
                                                  
    aa=bb=cc=dd=ee=0

    with torch.no_grad():
        prediction = loaded_model([img])    
                           
    for element in range(len(prediction[0]["scores"])):
        score = np.round(prediction[0]["scores"][element].numpy(), decimals= 4)
        
        if score > 0.5:
            if prediction[0]["labels"][element].numpy() == 1:
                aa=1
            elif prediction[0]["labels"][element].numpy() == 2:
                bb=1
            elif prediction[0]["labels"][element].numpy() == 3:
                cc=1
            elif prediction[0]["labels"][element].numpy() == 4:               
                dd=1
            elif prediction[0]["labels"][element].numpy() == 5:
                ee=1
        else:
            break
                           
    if aa == 1:
        class1.append(True)
    else:
        class1.append(False)

    if bb == 1:
        class2.append(True)
    else:
        class2.append(False)

    if cc == 1:
        class3.append(True)
    else:
        class3.append(False)

    if dd == 1:
        class4.append(True)
    else:
        class4.append(False)

    if ee == 1:
        class5.append(True)
    else:
        class5.append(False)
    
    files.append(imgs[i])
                           

pred_df = pd.DataFrame({'image_id':files,'d0':class1,'d1':class2, 'd2':class3,  'd3':class4, 'd4':class5 })

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.72s/it]


In [22]:
pred_df.to_pickle(r".\..\..\..\..\C2_TrainDev\N26091194_predict.pkl")
pred_df.to_csv(r".\..\..\..\..\C2_TrainDev\N26091194_predict.tsv", sep = '\t')
print('已輸出結果至C2_TrainDev資料匣，請前往察看')

已輸出結果至C2_TrainDev資料匣，請前往察看
